In [ ]:
#download and unzip data
%env KAGGLE_USERNAME=shivindass
%env KAGGLE_KEY=6930b85bc2a7e49c2bf390121bee0341

!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f customers.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f articles.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f transactions_train.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f sample_submission.csv

!unzip customers.csv.zip -d dataset
!unzip articles.csv.zip -d dataset
!unzip transactions_train.csv.zip -d dataset
!unzip sample_submission.csv.zip -d dataset

!rm customers.csv.zip articles.csv.zip transactions_train.csv.zip sample_submission.csv.zip

env: KAGGLE_USERNAME=shivindass
env: KAGGLE_KEY=6930b85bc2a7e49c2bf390121bee0341
 89% 87.0M/97.9M [00:00<00:00, 177MB/s]
100% 97.9M/97.9M [00:00<00:00, 170MB/s]
  0% 0.00/4.26M [00:00<?, ?B/s]
100% 4.26M/4.26M [00:00<00:00, 104MB/s]
100% 583M/584M [00:06<00:00, 116MB/s] 
100% 584M/584M [00:06<00:00, 98.1MB/s]
100% 50.3M/50.3M [00:00<00:00, 145MB/s]
100% 50.3M/50.3M [00:00<00:00, 125MB/s]
Archive:  customers.csv.zip
replace dataset/customers.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/customers.csv   
Archive:  articles.csv.zip
replace dataset/articles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#install implicit
!pip install implicit

import implicit
from scipy.sparse import csr_matrix
import pandas as pd
import tqdm

     |████████████████████████████████| 18.5 MB 464 kB/s 


/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [ ]:
#utilities
import os
data_path = "dataset/"
def load_csv(filename, dtype=str):
  '''
    load file from DATA_DIR/filename
  '''
  path = os.path.join(data_path, filename)

  if os.path.exists(path):
    return pd.read_csv(path, dtype=dtype)
  else:
    raise Exception(
        "Incorrect filepath: " + path
      )

def save_csv(data, filename):
  '''
    save data to DAT_DIR/filename
  '''
  path = os.path.join(data_path, filename)
  data.to_csv(path, index = False)


def get_train_data(cutoff_date = '2020-09-15'):
  '''
    generate training data by only returning the transactions that took place on or before 2020-09-15
  '''

  train = load_csv("transactions_train.csv")

  train.t_dat = pd.to_datetime(train.t_dat)
  if cutoff_date:
    return train.loc[train.t_dat <= pd.to_datetime(cutoff_date)]
  return train

def get_val_data(cutoff_date = '2020-09-15'):
  '''
    generate validation data by only returning the articles purchased in the week 2020-09-16 to 2020-09-22  
  '''

  val = load_csv("transactions_train.csv")
  sub = load_csv("sample_submission.csv")
  cid = pd.Data

  val.t_dat = pd.to_datetime(val.t_dat)
  val = val.loc[val.t_dat > pd.to_datetime(cutoff_date)]

  val = val.groupby('customer_id').article_id.apply(list).reset_index()
  val = val.rename({'article_id' : 'prediction'}, axis = 1)
  print(val.head)

  val['prediction'] = val.prediction.apply(lambda x: ' '.join([str(k) for k in x]))
  
  val.rename(columns={"Unnamed: 0": "customer_id"})

  for c in sample_sub['customer_id']:
    print(val[c])
  
  return val

'''
    Code taken from kaggle's github repository - https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
'''

import numpy as np

def apk(actual, predicted, k=12):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a.split(),p.split(),k) for a,p in zip(actual, predicted)])

In [ ]:
val = get_val_data(cutoff_date='2020-09-15')

<bound method NDFrame.head of                                              customer_id  \
0      00039306476aaf41a07fed942884f16b30abfa83a2a8be...   
1      0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...   
2      000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...   
3      000525e3fe01600d717da8423643a8303390a055c578ed...   
4      00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...   
...                                                  ...   
68979  fffa67737587e52ff1afa9c7c6490b5eb7acbc439fe82b...   
68980  fffa7d7799eb390a76308454cbdd76e473d65b1497fbe4...   
68981  fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde...   
68982  fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...   
68983  fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...   

                                              prediction  
0                                           [0624486001]  
1                                           [0827487003]  
2                   [0757926001, 0788575004, 0640021019]  
3            

In [ ]:
#Load train and val data
cutoff_date = '2020-09-15'
transactions = get_train_data(cutoff_date = cutoff_date)
val = get_val_data(cutoff_date = cutoff_date)

In [ ]:
article_n = 20000
customer_n = 20000

customer_counts = transactions['customer_id'].value_counts()
article_counts = transactions['article_id'].value_counts()

sub_sample = transactions[transactions['article_id'].isin(article_counts.index[:article_n]) & transactions['customer_id'].isin(customer_counts.index[:customer_n])]

article_to_index = {article_counts.index[i]: i for i in range(article_n)}
customer_to_index = {customer_counts.index[i]: i for i in range(customer_n)}

#create sparse matrix
art_per_cust = sub_sample[['article_id', 'customer_id']].value_counts()
max_art_per_cust = art_per_cust[0]

data = list(art_per_cust[:])/max_art_per_cust

row_ind = []
col_ind = []
for a, c in art_per_cust.index:
  row_ind.append(customer_to_index[c])
  col_ind.append(article_to_index[a])

user_item_matrix = csr_matrix((data, (row_ind, col_ind)), shape=(customer_n, article_n))

#train model
model = implicit.als.AlternatingLeastSquares(iterations=30, factors=128)
model.fit(user_item_matrix)

#make predictions
submission = load_csv("sample_submission.csv")

most_popular_pred = ' '.join(article_counts.index[:12])
predictions = []
for cust_id in tqdm.tqdm(submission["customer_id"]):
  if cust_id in customer_to_index.keys():
    cust_index = customer_to_index[cust_id]
    recommendations = model.recommend(userid = cust_index, user_items = user_item_matrix[cust_index], N=12, filter_already_liked_items=False)
    predicted_items = []
    for r in recommendations[0]:
      predicted_items.append(article_counts.index[r])
    predictions.append(' '.join(predicted_items))
  else:
    predictions.append(most_popular_pred)
# print(predictions)
submission['prediction'] = predictions

        t_dat                                        customer_id  article_id  \
84 2018-09-20  00609a1cc562140fa87a6de432bef9c9f0b936b259ad30...  0611415005   
85 2018-09-20  00609a1cc562140fa87a6de432bef9c9f0b936b259ad30...  0578374001   
86 2018-09-20  00609a1cc562140fa87a6de432bef9c9f0b936b259ad30...  0673677002   
87 2018-09-20  00609a1cc562140fa87a6de432bef9c9f0b936b259ad30...  0676352001   
88 2018-09-20  00609a1cc562140fa87a6de432bef9c9f0b936b259ad30...  0611415001   

                   price sales_channel_id  
84  0.016932203389830508                2  
85  0.042355932203389825                2  
86  0.016932203389830508                2  
87  0.025406779661016947                2  
88  0.016932203389830508                2  


In [ ]:
mapk(val['prediction'], submission['prediction'])

In [ ]:
# save_csv(submission, "submission.csv")
# !kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f dataset/submission.csv -m "Matrix fact"

100% 258M/258M [00:12<00:00, 21.7MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations